In [73]:
import importlib
import json
from pathlib import Path
from typing import Callable, Tuple

import n2g
import torch
import transformer_lens  # type: ignore[import]
from jaxtyping import Float, Int
from n2g import NeuronStats, Tokenizer
from torch import Tensor
from transformer_lens import HookedTransformer
from transformer_lens.hook_points import HookPoint  # type: ignore[import]

from thesis.device import Device
from thesis.mas.mas_store import MASStore
from thesis.sae.sae import SparseAutoencoder
from thesis.mas import html

importlib.reload(n2g)
importlib.reload(n2g.fit)
importlib.reload(n2g.neuron_model)


<module 'n2g.neuron_model' from 'C:\\Users\\alber\\Documents\\Neuron2Graph\\n2g\\neuron_model.py'>

In [74]:
FEATURES: list[int] = [0]

device = Device.get()

mas_store = MASStore.load(Path("outputs/models/gelu-1l-sae_store.zip"), device)

model = transformer_lens.HookedTransformer.from_pretrained("gelu-1l", device.torch())
sae = SparseAutoencoder.from_hf("NeelNanda/sparse_autoencoder", "25.pt", "blocks.0.mlp.hook_post", device)

tokenizer = Tokenizer(model)

assert mas_store.feature_samples().shape[0] == 2048 * 9


def feature_samples(feature_index: int) -> Tuple[list[str], float]:
    store_index = FEATURES[feature_index] + 2048

    samples = mas_store.feature_samples()[store_index, :, :]
    max_activation = mas_store.feature_max_activations()[store_index, :].max().item()

    tokens = ["".join(model.tokenizer.batch_decode(sample, clean_up_tokenization_spaces=False)) for sample in samples]

    return tokens, max_activation


with open("test.html", "w", encoding="utf-8") as f:
    html_str = html.generate_html(
        model, mas_store.feature_samples()[2048, :, :], mas_store.feature_activations()[2048, :, :]
    )
    f.write(html_str)


def model_feature_activation(
    model: HookedTransformer, layer_id: str, neuron_index: int
) -> Callable[[Int[Tensor, "num_samples sample_length"]], Float[Tensor, "num_samples sample_length"]]:
    def result(samples: Int[Tensor, "num_samples sample_length"]) -> Float[Tensor, "num_samples sample_length"]:
        activations: Float[Tensor, "num_samples sample_length"] = torch.full(samples.shape, float("nan"))

        def hook(activation: Float[Tensor, "num_samples sample_length neurons_per_layer"], hook: HookPoint) -> None:
            activations[:] = activation[:, :, neuron_index]

        with torch.no_grad():
            model.run_with_hooks(samples, fwd_hooks=[(layer_id, hook)])
            assert not torch.isnan(activations).any(), "Activations should not contain NaNs"

        return activations

    return result


def feature_activation(
    feature_index: int,
) -> Callable[[Int[Tensor, "num_samples sample_length"]], Float[Tensor, "num_samples sample_length"]]:
    return lambda samples: sae.feature_activations(model, samples, FEATURES[feature_index])


with Path("thesis/n2g/word_to_casings.json").open("r", encoding="utf-8") as f:
    word_to_casings = json.load(f)

train_config = n2g.TrainConfig(
    n2g.fit.FitConfig(n2g.fit.PruneConfig(batch_size=1), n2g.fit.ImportanceConfig(), n2g.fit.AugmentationConfig())
)

samples = feature_samples(0)[0]
tokenizer = n2g.Tokenizer(model)
tokens, str_tokens = tokenizer.batch_tokenize_with_str(samples, True)
tokens_non_bos, str_tokens = tokenizer.batch_tokenize_with_str(samples, False)
activations = sae.feature_activations(model, mas_store.feature_samples()[2048, :, :], 0)
tokenized_activations = sae.feature_activations(model, tokens, 0)
tokenized_activations_non_bos = sae.feature_activations(model, tokens_non_bos, 0)
mas_activations = mas_store.feature_activations()[2048, :, :]

with open("test2.html", "w", encoding="utf-8") as f:
    html_str = html.generate_html(model, mas_store.feature_samples()[2048, :, :], mas_activations)
    f.write(html_str)

with open("test3.html", "w", encoding="utf-8") as f:
    html_str = html.generate_html(model, mas_store.feature_samples()[2048, :, :], activations)
    f.write(html_str)

with open("test4.html", "w", encoding="utf-8") as f:
    html_str = html.generate_html(model, tokens, tokenized_activations)
    f.write(html_str)

with open("test5.html", "w", encoding="utf-8") as f:
    html_str = html.generate_html(model, tokens_non_bos, tokenized_activations_non_bos)
    f.write(html_str)


stats: list[NeuronStats]
models, stats = n2g.run_layer(
    len(FEATURES), feature_activation, feature_samples, tokenizer, word_to_casings, device.torch(), train_config
)


Loaded pretrained model gelu-1l into HookedTransformer
feature_index=0
Num train samples: 16
Processing 1 of 16
tensor([185])
num_pruned_results: 1
num augmented samples: 6
Processing 2 of 16
tensor([190])
num_pruned_results: 1
num augmented samples: 1
Processing 3 of 16
tensor([18])
num_pruned_results: 1
num augmented samples: 1
Processing 4 of 16
tensor([243])
num_pruned_results: 1
num augmented samples: 1
Processing 5 of 16
tensor([218])
num_pruned_results: 1
num augmented samples: 6
Processing 6 of 16
tensor([164])
num_pruned_results: 1
num augmented samples: 21
Processing 7 of 16
tensor([130])
num_pruned_results: 1
num augmented samples: 11
Processing 8 of 16
tensor([94])
num_pruned_results: 1
num augmented samples: 16
Processing 9 of 16
tensor([255])
num_pruned_results: 1
num augmented samples: 1
Processing 10 of 16
tensor([193])
num_pruned_results: 1
num augmented samples: 1
Processing 11 of 16
tensor([153])
num_pruned_results: 1
num augmented samples: 6
Processing 12 of 16
tens

In [75]:
index = 185
sample = feature_samples(0)[0][0]
tokenizer = n2g.Tokenizer(model)
tokens, str_tokens = tokenizer.tokenize_with_str(sample, True)
print(str_tokens[index - 5 : index + 5])
# feature_samples(0)[0]#[index]
print(sample)

feature_activation(0)(tokens).argmax()


['\n  ', ' cf', '_', 'en', 'viron', '.', 'execute', '()', '\n   ', '\n  ']

   cf.insert_link( "link_3",  "One_Step",       [ sprinkler_element.check_for_excessive_flow_rate ] )
   cf.insert_link( "link_3",  "Code",           [ sprinkler_element.monitor ] )
   cf.insert_link( "link_4",  "SendEvent",      ["CELL_DONE"] ) 
   cf.insert_link( "link_5",  "Disable_Chain",  [["monitor_irrigation_cell","monitor_current_sub" ]])


   length = redis.llen("QUEUES:SPRINKLER:IRRIGATION_CELL_QUEUE" )
   
   cf_environ = py_cf.Execute_Cf_Environment( cf )
   cf_environ.execute()
   
  

       
     

      

<|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD|><|PAD

tensor(13, device='cuda:0')

In [76]:
models[0].graphviz()